<a href="https://colab.research.google.com/github/Deon62/FIneTuned-EmotionDetectModel/blob/main/ModelFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate nlpaug


In [ ]:
from datasets import load_dataset
import pandas as pd
import nlpaug.augmenter.word as naw
from collections import Counter


In [ ]:

ds = load_dataset("dair-ai/emotion", "split")

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [ ]:
# Convert train split to DataFrame
train_df = pd.DataFrame(ds["train"])

# Display first few rows
print(train_df.head())

# Class distribution
counter = Counter(train_df["label"])
label_names = ds["train"].features["label"].names
for label_id, count in counter.items():
    print(f"{label_names[label_id]:<8}: {count}")


                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3
sadness : 4666
anger   : 2159
love    : 1304
surprise: 572
fear    : 1937
joy     : 5362


In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased",do_lower_case=True)

In [ ]:
max_length = 128

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_length
    )


In [ ]:
tokenized_datasets = ds.map(
    tokenize_batch,
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest"
)


In [ ]:
tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)
train_ds = tokenized_datasets["train"]
val_ds   = tokenized_datasets["validation"]
test_ds  = tokenized_datasets["test"]


In [ ]:
from transformers import (
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)
import evaluate


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,                # our 6 emotion classes
    ignore_mismatched_sizes=True # safe if vocab sizes differ
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric       = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    f1  = f1_metric.compute(predictions=preds, references=labels, average="macro")
    return {"accuracy": acc["accuracy"], "f1_macro": f1["f1"]}


In [ ]:
training_args = TrainingArguments(
    output_dir="emotion-distilbert",
    do_train=True,
    do_eval=True,
    eval_steps=500,
    save_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=50,
    fp16=True,
    gradient_checkpointing=False
)


In [ ]:
!pip install peft


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,

    # Tell PEFT which linear modules to wrap with LoRA
    target_modules=["q_lin", "v_lin"]  # for DistilBERT’s attention projections
)
model = get_peft_model(model, lora_config)


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total params: {total_params:,}")
print(f"Trainable params: {trainable_params:,}")  # should be just the LoRA adapters


Total params: 67,700,748
Trainable params: 742,662


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                # your LoRA-wrapped model
    args=training_args,         # the args that worked for you
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-55-77475f9956c7>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
train_result = trainer.train()


Step,Training Loss
50,1.805000
100,1.752200
150,1.662000
200,1.564500
250,1.492900
300,1.329300
350,1.243000
400,1.117000
450,0.982600
500,0.940800


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd

# 1. Get predictions on the test split
preds_output = trainer.predict(test_ds)
pred_labels = np.argmax(preds_output.predictions, axis=-1)
true_labels = preds_output.label_ids

# 2. Map numeric IDs to emotion names
label_names = ds["train"].features["label"].names

# 3. Confusion Matrix
cm = confusion_matrix(true_labels, pred_labels)
cm_df = pd.DataFrame(cm, index=label_names, columns=label_names)
print("Confusion Matrix:\n", cm_df)

# 4. Classification Report
cr = classification_report(true_labels, pred_labels, target_names=label_names)
print("\nClassification Report:\n", cr)


Confusion Matrix:
           sadness  joy  love  anger  fear  surprise
sadness       544   11     4     17     5         0
joy            12  644    27      6     3         3
love            5   33   117      3     1         0
anger          19    4     3    244     5         0
fear           11    3     0     10   185        15
surprise        3    7     0      1     9        46

Classification Report:
               precision    recall  f1-score   support

     sadness       0.92      0.94      0.93       581
         joy       0.92      0.93      0.92       695
        love       0.77      0.74      0.75       159
       anger       0.87      0.89      0.88       275
        fear       0.89      0.83      0.86       224
    surprise       0.72      0.70      0.71        66

    accuracy                           0.89      2000
   macro avg       0.85      0.83      0.84      2000
weighted avg       0.89      0.89      0.89      2000



In [ ]:
model.save_pretrained("Deon_emotion-model")
tokenizer.save_pretrained("Deon-emotion-model")


('Deon-emotion-model/tokenizer_config.json',
 'Deon-emotion-model/special_tokens_map.json',
 'Deon-emotion-model/vocab.txt',
 'Deon-emotion-model/added_tokens.json',
 'Deon-emotion-model/tokenizer.json')

In [74]:
# from huggingface_hub import login
# login()



In [ ]:
model.push_to_hub("chinesemusk/Deon_emotion-model")
tokenizer.push_to_hub("chinesemusk/Deon-emotion-model")

adapter_model.safetensors:   0%|          | 0.00/2.97M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chinesemusk/Deon-emotion-model/commit/efb9109f1e98dcc94c5d3b8b0fc658e8aaccc48e', commit_message='Upload tokenizer', commit_description='', oid='efb9109f1e98dcc94c5d3b8b0fc658e8aaccc48e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chinesemusk/Deon-emotion-model', endpoint='https://huggingface.co', repo_type='model', repo_id='chinesemusk/Deon-emotion-model'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
    pipeline
)
from peft import PeftModel

# 1. Load the base model with the correct label count
base = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6
)

# 2. Attach your LoRA adapters from your Hub repo
model = PeftModel.from_pretrained(
    base,
    "chinesemusk/Deon_emotion-model"  # your model repo name
)

# 3. Load the matching tokenizer (from the same repo)
tokenizer = DistilBertTokenizerFast.from_pretrained(
    "chinesemusk/Deon-emotion-model"
)

# 4. Build the pipeline on that exact model+tokenizer
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

# 5. Test it!
print(classifier("I just got promoted at work — I'm over the moon!"))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/2.97M [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DiffLlamaForSequenceClass

[[{'label': 'LABEL_0', 'score': 0.025199497118592262}, {'label': 'LABEL_1', 'score': 0.9219207167625427}, {'label': 'LABEL_2', 'score': 0.014422563835978508}, {'label': 'LABEL_3', 'score': 0.02757374383509159}, {'label': 'LABEL_4', 'score': 0.005670473445206881}, {'label': 'LABEL_5', 'score': 0.005213005933910608}]]


In [68]:
import torch
import torch.nn.functional as F

# 1. Put model in eval mode & move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).eval()

# 2. Define label names
label_names = ds["train"].features["label"].names

def predict_emotion(text: str):
    # Tokenize & convert to tensors
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    # Forward pass (LoRA adapters are active)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Softmax → probabilities
    probs = F.softmax(logits, dim=-1).squeeze().cpu().tolist()

    # Pair labels & scores
    return dict(zip(label_names, probs))

# 3. Try it out
print(predict_emotion("I can't believe how beautiful the sunset is!"))


{'sadness': 0.025863826274871826, 'joy': 0.8155686259269714, 'love': 0.10546589642763138, 'anger': 0.00277561042457819, 'fear': 0.006003082729876041, 'surprise': 0.04432295635342598}


In [69]:
merged_model = model.merge_and_unload()
merged_model.push_to_hub("chinesemusk/Deon_emotion-model-full")
tokenizer.push_to_hub("chinesemusk/Deon-emotion-model-full")


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/chinesemusk/Deon-emotion-model-full/commit/0adc325722acb9d2e23f618efc7956fc5ab79b0e', commit_message='Upload tokenizer', commit_description='', oid='0adc325722acb9d2e23f618efc7956fc5ab79b0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/chinesemusk/Deon-emotion-model-full', endpoint='https://huggingface.co', repo_type='model', repo_id='chinesemusk/Deon-emotion-model-full'), pr_revision=None, pr_num=None)

In [70]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="chinesemusk/Deon_emotion-model-full",
    tokenizer="chinesemusk/Deon-emotion-model-full",
    top_k=None  # returns all class scores
)

text = "I'm feeling really hopeful and excited today!"
results = classifier(text)
print(results)


Device set to use cuda:0


[[{'label': 'LABEL_1', 'score': 0.9955287575721741}, {'label': 'LABEL_2', 'score': 0.001875934423878789}, {'label': 'LABEL_5', 'score': 0.0008058727835305035}, {'label': 'LABEL_4', 'score': 0.0007722158916294575}, {'label': 'LABEL_0', 'score': 0.0006365749868564308}, {'label': 'LABEL_3', 'score': 0.0003806230379268527}]]


In [71]:
!pip install gradio
import gradio as gr
from transformers import pipeline

# Load your model and tokenizer from Hugging Face
classifier = pipeline(
    "text-classification",
    model="chinesemusk/Deon_emotion-model-full",
    tokenizer="chinesemusk/Deon-emotion-model-full",
    top_k=None  # to show scores for all classes
)

# Define the interface function
def classify_emotion(text):
    results = classifier(text)[0]
    # Format results for display
    return {res['label']: float(f"{res['score']:.3f}") for res in results}

# Launch the Gradio UI
gr.Interface(
    fn=classify_emotion,
    inputs=gr.Textbox(lines=2, placeholder="Enter a sentence..."),
    outputs=gr.Label(num_top_classes=6),
    title="Deon Emotion Classifier",
    description="Enter a sentence to detect the expressed emotion (joy, love, sadness, anger, fear, surprise)."
).launch()


Device set to use cuda:0


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a9898c1c6e6d9b3da1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [73]:
# app.py
import gradio as gr
from transformers import pipeline

# Load pipeline from your fine-tuned model
classifier = pipeline(
    "text-classification",
    model="chinesemusk/Deon_emotion-model-full",
    tokenizer="chinesemusk/Deon-emotion-model-full",
    top_k=None,  # Show all class probabilities
    device=0  # Use GPU if available
)

# Define prediction function
def predict_emotion(text):
    results = classifier(text)[0]
    # Format as label: probability%
    return {res["label"]: float(f"{res['score']:.4f}") for res in results}

# Build Gradio interface
demo = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Textbox(lines=3, placeholder="Enter text here..."),
    outputs=gr.Label(num_top_classes=6),
    title="Deon Emotion Classifier",
    description="Predicts emotion from English text using a fine-tuned DistilBERT model. Try inputs like 'I love this moment' or 'I'm scared.'",
    examples=[
        ["I love this!"],
        ["I feel so empty today..."],
        ["This is amazing!"],
        ["I'm really angry right now"],
        ["What a shocking turn of events!"]
    ]
)

if __name__ == "__main__":
    demo.launch()


config.json:   0%|          | 0.00/737 [00:00<?, ?B/s]

Device set to use cuda:0


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://de95664a4b955bea27.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
